In [27]:
import pandas as pd
import folium
import joblib

model = joblib.load('../models/model_09v4.pkl')


In [28]:

df = pd.read_csv('../data/homicide_cleaned.csv', parse_dates=['Incident_Date'])
df = df.dropna(subset=['Latitude', 'Longitude', 'Weapon'])





In [29]:
def predict_weapon(df_filtered):
    X_pred = df_filtered.drop(['Incident_Date', 'Weapon', 'Gender', 'Race',
                               'Weapon_Simplified'], axis=1, errors='ignore')
    return model.predict(X_pred)



In [30]:
df_subset = df[df['Year'] == 2023]

In [31]:
df_subset['Predicted_Weapon'] = predict_weapon(df_subset)

/var/folders/_b/jzznzsb926918cg1p5rj5vfw0000gn/T/ipykernel_38883/2885362606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Predicted_Weapon'] = predict_weapon(df_subset)


In [32]:
map = folium.Map(location=[df_subset['Latitude'].mean(), df_subset['Longitude'].mean()], zoom_start=11)

for _, row in df_subset.iterrows():
    popup_text = f"""
    True: {row['Weapon_Simplified']}<br>
    Predicted: {row['Predicted_Weapon']}'
    """
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map)

map